In [1]:
import sys
sys.path.append("../")

In [2]:
import systemData
import cell
import klass
import spot
import utils
import config
import logging

2019-03-04 17:12:26,095:DEBUG:CACHEDIR=C:\Users\experiment\.matplotlib
2019-03-04 17:12:26,102:DEBUG:Using fontManager instance from C:\Users\experiment\.matplotlib\fontList.json
2019-03-04 17:12:26,697:DEBUG:backend module://ipykernel.pylab.backend_inline version unknown
2019-03-04 17:12:27,528:DEBUG:backend module://ipykernel.pylab.backend_inline version unknown


In [3]:
CONFIG_FILE = '../src/config.yml'

logger = logging.getLogger()
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s:%(levelname)s:%(message)s"
    )

In [4]:
ini = config.read(CONFIG_FILE, 'DEFAULT')

# make a cell object
cells = cell.Cell(ini)

# make a spots object
spots = spot.Spot(ini)

# make a klass object
klasses = klass.Klass(ini['gSet'])

# calc the loglik and populate some of the object's properties
spots.loglik(cells, ini)

# make now a genes object
genes = spots.getGenes()

# universe
gSub = ini['gSet'].GeneSubset(genes.names)
gSub = gSub.ScaleCell(0)

# now you can set expressions and logexpressions (as the mean expression over klass)
genes.setKlassExpressions(klasses, ini, gSub)


2019-03-04 17:12:27,561:INFO:reading CellMap from ../data/default/CellMap.mat
2019-03-04 17:12:32,491:INFO:Rebasing CellYX to match the one-based indexed Matlab object. 
2019-03-04 17:12:32,494:INFO:********* Getting spotattributes from ../data_preprocessed/spot_attributes.nc **********
2019-03-04 17:12:32,731:INFO:Rebasing SpotYX to match the one-based indexed Matlab object. Not sure this is needed!!
2019-03-04 17:12:32,732:INFO:Rebasing Neighbors to match the one-based indexed Matlab object. Not sure this is needed! REMOVE IT
..\src\spot.py:84: RuntimeWarning: invalid value encountered in greater
  sanity_check = Neighbors[SpotInCell > 0, 0] + 1 == SpotInCell[SpotInCell > 0]
..\src\spot.py:87: RuntimeWarning: invalid value encountered in greater
  D[SpotInCell > 0, 0] = D[SpotInCell > 0, 0] + ini['InsideCellBonus']


In [5]:
p0 = None
for i in range(ini['CellCallMaxIter']):
    # calc the number of copies of each gene in each cell
    cells.geneCount(spots, genes)
    
    # cell calling: Assign cells to klasses
    cells.klassAssignment(spots, genes, klasses, ini)

    # spot calling: Assign spots to cells
    spots.cellAssignment(cells, genes, klasses)

    # Update parameter
    genes.updateGamma(cells, spots, klasses, ini)

    converged, delta = src.utils.isConverged(spots, p0, ini['CellCallTolerance'])
    logger.info('Iteration %d, mean prob change %f' % (i, delta))

    # replace p0 with the latest probabilities
    p0 = spots.neighbors['prob']

    if converged:
        print("Success!!")
        break


print("done")

2019-03-04 17:12:35,457:INFO:cell ---> klass probabilities updated
2019-03-04 17:12:36,267:INFO:spot ---> cell probabilities updated
2019-03-04 17:12:36,493:INFO:Iteration 0, mean prob change 1.000000
2019-03-04 17:12:37,342:INFO:cell ---> klass probabilities updated
2019-03-04 17:12:38,162:INFO:spot ---> cell probabilities updated
2019-03-04 17:12:38,381:INFO:Iteration 1, mean prob change 0.794103
2019-03-04 17:12:39,206:INFO:cell ---> klass probabilities updated
2019-03-04 17:12:40,015:INFO:spot ---> cell probabilities updated
2019-03-04 17:12:40,230:INFO:Iteration 2, mean prob change 0.465452
2019-03-04 17:12:41,066:INFO:cell ---> klass probabilities updated
2019-03-04 17:12:41,873:INFO:spot ---> cell probabilities updated
2019-03-04 17:12:42,089:INFO:Iteration 3, mean prob change 0.271388
2019-03-04 17:12:42,916:INFO:cell ---> klass probabilities updated
2019-03-04 17:12:43,734:INFO:spot ---> cell probabilities updated
2019-03-04 17:12:43,969:INFO:Iteration 4, mean prob change 0.22

Success!!
done
